#### Week 1. Titanic Classification with DistilBERT + XGBoost

`Titanic: Machine Learning from Disaster`은 머신러닝을 처음 공부했을 때 다들 한번씩은 거쳐갔을 유명한 실습예제 중 하나입니다.

이번 주차 학습 목표는 다음과 같습니다.


1. 온라인 공모전 사이트 `kaggle`에서 데이터를 받고, 모델로 학습한 결과를 csv파일로 업로드하기

2. 자연어처리 모델인 BERT 모델을 Huggingface에서 받아와 분류하기 전 데이터를 숫자 벡터로 Embedding하기

3. Embedding한 숫자 벡터를 머신러닝 알고리즘으로 분류작업 진행하기

Kaggle Titanic Competition 바로가기 : https://www.kaggle.com/competitions/titanic  

### 1. Kaggle에서 데이터 받아오기

In [ ]:
## pip install ~ : 패키지 다운로드
## -q : 로그 메세지 출력 X

!pip install -q kaggle

In [ ]:
## Kaggle에서 데이터를 받아오기 위해서는 Authenticator Token인 "kaggle.json"이 있어야 합니다.
## 자세한 내용은 영상을 참고하세요(kaggle 사이트의 settings > create new tokens로 다운받을 수 있다.)

from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Kaggle 라이브러리를 어떻게 사용하고 연동하려면 필요한 작업이다.
# 위에서 다운로드 받은 kaggle.json파일을 인식할 수가 없어서 아래 방법을 진행해준다.

# 1. root경로에 .kaggle 파일 제작

!mkdir ~/.kaggle

# 자세한 의미는 아래와 같다.

In [ ]:
# '~'은 root를 의미한다.
# root는 말 그자체로 뿌리가 되는 경로이다.(이 경로를 통해 절대경로를 지정할 수 있다.)
# 그러나 root경로가 가장 상위의 경로인 것은 아니다.

# 가장 상위의 경로는 '/'이다.
# 여기 안에 content와 root폴더들이 들어가있다.
%cd ..
print('\n')
!ls

/


bin	 dev   lib32   mnt			 python-apt  srv    usr
boot	 etc   lib64   NGC-DL-CONTAINER-LICENSE  root	     sys    var
content  home  libx32  opt			 run	     tmp
datalab  lib   media   proc			 sbin	     tools


In [ ]:
# 1. '~'은 현재 나의 위치와 무관하게 root폴더의 경로를 가르킨다.

%cd /content
%cd ~
!pwd # 즉, 현재 경로가 '/content'이어도 동일위치에 있는 /root폴더로 이동이 가능하다.

/content
/root
/root


In [ ]:
# 2. 그렇다고 '~'이 root와 같은 것은 아니다. 하나의 기능이라고 보면 된다.

# 가장상위경로인 '/'에서 root폴더안으로 들어간다. (주의) 가끔씩 %cd 뒤에 #을 해도 주석으로 받아들이지 못해 오류가 나기도 해서, 이렇게 따로 주석을 다른 줄에 작성해주자.
%cd ..
%cd /root

# 어떤 위치에서든 root폴더안으로 들어간다.
%cd ..
%cd ~

# '~'를 쓸 때는 가장 앞에 써줘야한다. 아니면 오류가 발생한다.
%cd ..
%cd /~

/
/root
/
/root
/
[Errno 2] No such file or directory: '/~'
/


In [ ]:
# 3. '~'를 통해 절대경로를 사용할 수 있다.

# case1. 현재경로를 모르나 content폴더안으로 들어가고 싶을 때
%cd ~/../content

# '~'을 통해 어디서든 root폴더안으로 들어가서 그 상위폴더인 '/'으로 들어간다.(..) 이후에 content폴더안으로 들어갈 수 있다.

/content


In [ ]:
# 다시 본론으로 돌아가서 root에 생성된 .kaggle 파일을 찾을 수 있는지 보자.

%cd ~
!ls -a # root 안에 있는 .으로 확장자로 되어있어 '-a'를 통해 확인이 가능하다. 여기에 .kaggle파일이 생성된 것이 확인된다.

/root
.   .bashrc  .config   .jupyter  .keras  .npm	   .tmux.conf
..  .cache   .ipython  .kaggle	 .local  .profile  .wget-hsts


In [ ]:
# 다시 원래 content 경로로 들어오자.

%cd ~/../content

/content


In [ ]:
%cd /content/drive/MyDrive/실데사
!ls

/content/drive/MyDrive/실데사
kaggle.json  week1


In [ ]:
%cd ~/../content/drive/MyDrive/실데사
# 위에랑 같은 경로이다.
!ls

/content/drive/MyDrive/실데사
kaggle.json  week1


In [ ]:
## Drive에 kaggle.json을 업로드한 경로를 적으시면 됩니다.

# cp : copy & paste
# cp a경로 b경로 : a경로에 있는 걸, b경로로 복사붙여넣기한다.

!cp /content/drive/MyDrive/실데사/kaggle.json ~/.kaggle/

In [48]:
# 잘 붙여넣기가 되었는지 확인해보자.

%cd ~/.kaggle
!ls # .kaggle 아래에 kaggle.json이 들어가있다.

/root/.kaggle
kaggle.json


In [49]:
# chmod는 어떤 파일에 대한 권한을 어떻게 설정할 것인가를 나타내는 명령어이다.
# chmod 다음에 '6' '0' '0'이 왔는데, 차례대로 나/그룹/전체 에 대해 권한을 어떻게 설정하는가를 의미한다.
# read(4), write(2), execute(1)이기에 read만 가능하면 4, read와 write가 가능하면 4+2=6으로 가능한 것들의 합으로 나타낸다.
# 여기서 사용한 chmod 600의 경우, 나에게만 read, write권한을 주겠다는 의미이다.

!chmod 600 ~/.kaggle/kaggle.json

In [50]:
## 현재 kaggle에서 진행중인 공모전 리스트를 확인할 수 있습니다.
## 저희는 https://www.kaggle.com/competitions/titanic 예제를 사용하겠습니다.

!kaggle competitions list

ref                                                                                           deadline             category            reward  teamCount  userHasEntered  
--------------------------------------------------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
https://www.kaggle.com/competitions/asl-fingerspelling                                        2023-08-24 23:59:00  Research          $200,000        705           False  
https://www.kaggle.com/competitions/icr-identify-age-related-conditions                       2023-08-10 23:59:00  Featured           $60,000       5282           False  
https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries                      2023-10-11 23:59:00  Featured           $60,000        116           False  
https://www.kaggle.com/competitions/hubmap-hacking-the-human-vasculature                      2023-07-31 23:59:00  Research           $50,000    

In [52]:
# kaggle라이브러리를 사용하려면, 지금 경로를 '/content'로 해야한다.
# 그리고 다운로드할 파일은 그냥 이름만 쓰면 된다.

!pwd
# 현재 경로가 /content가 아니니 바꿔준다.
%cd ~/../content

/root/.kaggle
/content


In [54]:
## Titanic 공모전 데이터를 코랩으로 받고, zip 압축 해제하기

# 공모전 이름
competition_name = "titanic"

# 공모전 다운로드 to local environment
! kaggle competitions download {competition_name} # {}를 쓰면 문자열을 경로로 들어가도록 바꿔준다.

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)


In [55]:
# {competition_name}이름의 폴더에 zip 파일 압축해제
! unzip {competition_name + ".zip"} -d {competition_name}

# 드라이브 확인을 완료했으므로 드라이브 mount를 해제합니다.(이제 구글드라이브 연동을 해제한다.)
drive.flush_and_unmount()

Archive:  titanic.zip
  inflating: titanic/gender_submission.csv  
  inflating: titanic/test.csv        
  inflating: titanic/train.csv       


### 2. 데이터 전처리

In [56]:
import numpy as np
import pandas as pd

train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [57]:
train.tail(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [58]:
train.shape

(891, 12)

In [59]:
from sklearn.model_selection import train_test_split

X = train.drop(columns = ['PassengerId', 'Survived'], axis = 1)
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.05, random_state = 77, stratify = y)
# 0.05로 validation을 하는 이유는 머신러닝이라서 크게 과적합을 걱정할 필요가 없기 때문이다.
# 그래서 좀 적게 validation을 했다.
# 보통은 0.1 ~ 0.2 정도로 하며, 만 개 이상의 데이터가 validation에 들어가도록 한다.

In [60]:
## GPU 활성화
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 3. 데이터 임베딩

In [61]:
from typing import List
from tqdm.notebook import tqdm

!pip install -q transformers
from transformers import AutoModel, AutoTokenizer

class Encode_with_BERT:
    def __init__(self):
        ## Huggingface에서 BERT 모델을 받아옵니다.
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased').to(device) # gpu환경에 모델을 보낸다.

    ## 입력한 `texts`을 768 길이의 숫자벡터로 특징을 추출합니다.
    def extract(self, texts: List[str]): # ':' <- indicator로 이러한 형태의 데이터가 들어와야함을 나타내는 것이다.
        features = np.zeros((len(texts), 768), dtype = np.float16) # 891 x 768 짜리 0으로 만들어서, 나중에 더해줘서 만들기 위함이다. 보통 BERT는 Embedding하는 vector의 크기가 768이다.
        for index, text in enumerate(tqdm(texts)): # tqdm : 진행되는 과정을 보이게하는 함수
            tokenized_text = self.tokenizer(text, return_tensors="pt").to(device) # gpu환경에 데이터도 보낸다.(모델은 미리 gpu환경에 가있다.)
            model_output = self.model(**tokenized_text)[0].detach().cpu() # gpu환경에서 계산을 한 뒤에, cpu환경으로 결과값을 보낸다.
            features[index, :] = model_output.numpy().mean(axis=1) # features에 더해서 업데이트를 한다.

        return features

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.0 MB/s eta 0:00:00


### 4. 임베딩한 데이터 머신러닝 알고리즘으로 분류

In [62]:
from xgboost.sklearn import XGBClassifier # XGBoost 말고 LightGBM... 등의 classification 모델을 사용해보자.
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

extractor = Encode_with_BERT()
scaler = StandardScaler()
classifier = XGBClassifier(use_label_encoder = False)

## XGBoost 학습
train_labels = [int(_) for _ in y_train.values] ## [0,1,1, ...]
texts = [", ".join(str(_)) for _ in X_train.values] ##[[0,1,male],[2,1,female]...]
train_features = scaler.fit_transform(extractor.extract(texts)) ## to (len(texts), 768) 숫자벡터
classifier.fit(train_features, train_labels) ## XGBoost train

## Prediction with Test Data
answer = [int(_) for _ in y_val.values]
texts = [", ".join(str(_)) for _ in X_val.values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

## 모델 예측의 정밀도 측정!
accuracy = accuracy_score(answer, preds)
print(f'\naccuracy : {accuracy*100:.2f}%')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


  0%|          | 0/846 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]


accuracy : 86.67%


### 5. csv로 저장 + kaggle에 submit

In [63]:
import csv

ids = test['PassengerId'].values
texts = [", ".join(str(_)) for _ in test.iloc[:, 1:].values]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

with open('submission.csv', "w") as to_file:
    csvwriter = csv.writer(to_file, delimiter=",", quotechar='"')
    csvwriter.writerow(["PassengerId", "Survived"])
    for id, pred in zip(ids, preds):
        csvwriter.writerow([id, pred])

  0%|          | 0/418 [00:00<?, ?it/s]

In [64]:
sub = pd.read_csv('submission.csv')
sub.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [65]:
# 캐글 상에 업로드 하면, 캐글 안에 데이터로 평가를 해준다.